In [ ]:
import pandas as pd

# Import dataset
data = pd.read_csv("survey.csv")
data.head()

#
# TODO: Investigate data set
#

In [ ]:
# Data Cleaning 1: Remove unecessary fields

# Clean column names
data.columns = ['timestamp', 'age', 'industry', 'title', 'title_context', 'salary', 'bonus', 'currency', 'currency_other', 'income_context',
                'country', 'us_state', 'city', 'years_experience', 'years_in_field', 'education', 'gender', 'race']

#
# TODO: remove following columns: 'timestamp', 'title', 'title_context', 'income_context', 'bonus'.
#
# hint: use data.drop()
#

data = # ...

In [ ]:
# Data Cleaning 2: Normalize values

# Turn salary (string) into float values
data["salary"] = data["salary"].astype(str).str.replace(",", "").astype(float)
data['salary'].mean()

# Map ranges to mean values
years_conversion = {'1 year or less': 0.5, '2 - 4 years': 3, '5-7 years': 6, '8 - 10 years': 9, '11 - 20 years': 15.5, '21 - 30 years': 25.5, '31 - 40 years': 35.5, '41 years or more': 47}

#
# TODO: Complete age_conversions by mapping age ranges to float values
#
# hint: similar to years_conversion
#

age_conversion = {
    '18-24': # ...
    ,'25-34': # ...
    ,'35-44': 
    ,'45-54': 
    ,'55-64': 
    ,'65 or over': 
    ,'under 18': 
}


# Apply mappings to convert values
data["years_experience"] = data["years_experience"].replace(years_conversion)
data["years_in_field"] = data["years_in_field"].replace(years_conversion)
data["age"] = data["age"].replace(age_conversion)

data.head()

#
# THINK why might we want to clean these values?
#

In [ ]:
industry_counts = data.value_counts('industry')
industry_counts.head(60)

In [63]:
# Data Cleaning 3: More Mappings!

industry_mapping = {
    "Computing or Tech": "Computing or Tech",
    "Education (Higher Education)": "Education",
    "Nonprofits": "Nonprofits",
    "Health care": "Health care",
    "Government and Public Administration": "Government and Public Administration",
    "Accounting, Banking & Finance": "Accounting, Banking & Finance",
    "Engineering or Manufacturing": "Engineering or Manufacturing",
    "Marketing, Advertising & PR": "Marketing, Advertising & PR",
    "Law": "Law",
    "Business or Consulting": "Business or Consulting",
    "Education (Primary/Secondary)": "Education",
    "Media & Digital": "Marketing, Advertising & PR",
    "Insurance": "Accounting, Banking & Finance",
    "Retail": "Business or Consulting",
    "Recruitment or HR": "Business or Consulting",
    "Property or Construction": "Engineering or Manufacturing",
    "Art & Design": "Marketing, Advertising & PR",
    "Utilities & Telecommunications": "Engineering or Manufacturing",
    "Transport or Logistics": "Engineering or Manufacturing",
    "Sales": "Business or Consulting",
    "Social Work": "Nonprofits",
    "Hospitality & Events": "Business or Consulting",
    "Entertainment": "Marketing, Advertising & PR",
    "Agriculture or Forestry": "Engineering or Manufacturing",
    "Leisure, Sport & Tourism": "Business or Consulting",
    "Publishing": "Marketing, Advertising & PR",
    "Library": "Education",
    "Libraries": "Education",
    "Biotech": "Health care",
    "Law Enforcement & Security": "Government and Public Administration",
    "Public Library": "Education",
    "Research": "Health care",
    "Manufacturing": "Engineering or Manufacturing",
    "Pharmaceuticals": "Health care",
    "Architecture": "Engineering or Manufacturing",
    "Real Estate": "Business or Consulting",
    "Pharmaceutical": "Health care",
    "Public Libraries": "Education",
    "Pharma": "Health care",
    "Biotechnology": "Health care",
    "Energy": "Engineering or Manufacturing",
    "Environmental Consulting": "Engineering or Manufacturing",
    "Science": "Health care",
    "Scientific Research": "Health care",
    "Translation": "Marketing, Advertising & PR",
    "Veterinary": "Health care",
    "Oil & Gas": "Engineering or Manufacturing",
    "Politics": "Government and Public Administration",
    "Automotive": "Engineering or Manufacturing",
    "Oil and Gas": "Engineering or Manufacturing",
    "Philanthropy": "Nonprofits",
    "Biotech/Pharma": "Health care",
    "Environmental": "Engineering or Manufacturing",
    "Museum": "Education",
    "Biomedical Research": "Health care",
    "Construction": "Engineering or Manufacturing"
}

# Filter out rows where 'currency' is 'AUD/NZD'
data = data[data["currency"] != "AUD/NZD"]
data = data[data["industry"].isin(industry_mapping.keys())]
data["industry"] = data["industry"].replace(industry_mapping)


In [ ]:
# Data Cleaning 4: Handling currency and currency_other

# Combine the value counts from both 'currency' and 'currency_other' columns
combined_currency_counts = pd.concat([data["currency"].value_counts(), data["currency_other"].value_counts()])

# Filter out USD and consider only currencies that appear more than once
non_usd_currencies = combined_currency_counts[combined_currency_counts > 1].index

# Convert to a list, excluding 'USD'
non_usd_currencies_list = [currency for currency in non_usd_currencies if currency != 'USD']
non_usd_currencies_list = list(set(non_usd_currencies_list))

# Print the list of non-USD currencies
print(non_usd_currencies_list)

# Combine currency and currency_other
data['currency'] = data['currency_other'].combine_first(data['currency'])
data = data.drop(columns=['currency_other'])

In [ ]:
import requests

# API URL to get exchange rates with USD as the base currency
api_key = "040531bc8bf5d856af6c4f1f"
exchange_rate_api_link = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/USD"

# Make the API request
response = requests.get(exchange_rate_api_link)
api_data = response.json()  # Convert response to JSON

# Extract conversion rates
conversion_rates = api_data.get("conversion_rates", {})

# List of non-USD currencies from your dataset (replace with actual list)

# Store exchange rates for relevant currencies in a dictionary
exchange_rates_dict = {currency: conversion_rates.get(currency) for currency in non_usd_currencies_list if currency in conversion_rates}

# Print the dictionary of exchange rates
print(exchange_rates_dict)

# NOTE: ONLY RUN THIS CELL ONCE

In [ ]:
#
# TODO: How many Hong Kong Dollars (HKD) can be converted from 57.34 USD?
#
# What does each value represent?
#

print()

#
# TODO: Which currency has the largest number conversion rate?
#
# Hint: .items() can be used
# it is NOT ZAR
#

print()



In [ ]:
# Apply conversion to USD
data["salary_usd"] = data.apply(lambda row: row["salary"] / exchange_rates_dict.get(row["currency"], 1), axis=1)

#
# TODO: Compare the median salary to the median USD salary. 
#

print()

In [ ]:
# Data cleaning
data["us_state"] = data["us_state"].fillna("")
data["city"] = data["city"].fillna("")
data["country"] = data["country"].fillna("")
country_dict = {
    "Australia": "Australia",
    "Australia ": "Australia",
    "Austria": "Austria",
    "Argentina": "Argentina",
    "Belgium": "Belgium",
    "Belgium ": "Belgium",
    "Bermuda": "Bermuda",
    "Brazil": "Brazil",
    "Bulgaria": "Bulgaria",
    "Canada": "Canada",
    "Canada ": "Canada",
    "Canda": "Canada",
    "Chile": "Chile",
    "China": "China",
    "Colombia": "Colombia",
    "Croatia": "Croatia",
    "Czech Republic": "Czech Republic",
    "Cyprus": "Cyprus",
    "Denmark": "Denmark",
    "Denmark ": "Denmark",
    "Danmark": "Denmark",
    "England": "United Kingdom",
    "England ": "United Kingdom",
    "England, UK": "United Kingdom",
    "England, United Kingdom": "United Kingdom",
    "Estonia": "Estonia",
    "Finland": "Finland",
    "finland": "Finland",
    "France": "France",
    "Germany": "Germany",
    "Germany ": "Germany",
    "Ghana": "Ghana",
    "Greece": "Greece",
    "Great Britain": "United Kingdom",
    "Great Britain ": "United Kingdom",
    "Hong Kong": "Hong Kong",
    "Hungary": "Hungary",
    "India": "India",
    "Indonesia": "Indonesia",
    "Ireland": "Ireland",
    "Ireland ": "Ireland",
    "Israel": "Israel",
    "Italy": "Italy",
    "Japan": "Japan",
    "Japan ": "Japan",
    "Kenya": "Kenya",
    "Latvia": "Latvia",
    "Lithuania": "Lithuania",
    "Luxembourg": "Luxembourg",
    "Malaysia": "Malaysia",
    "Mexico": "Mexico",
    "Morocco": "Morocco",
    "Netherlands": "Netherlands",
    "Netherlands ": "Netherlands",
    "The Netherlands": "Netherlands",
    "The Netherlands ": "Netherlands",
    "New Zealand": "New Zealand",
    "New Zealand ": "New Zealand",
    "NZ": "New Zealand",
    "Nigeria": "Nigeria",
    "Nigeria ": "Nigeria",
    "Norway": "Norway",
    "Norway ": "Norway",
    "Pakistan": "Pakistan",
    "Philippines": "Philippines",
    "Poland": "Poland",
    "Portugal": "Portugal",
    "Puerto Rico": "Puerto Rico",
    "Romania": "Romania",
    "Saudi Arabia": "Saudi Arabia",
    "Scotland": "United Kingdom",
    "Scotland ": "United Kingdom",
    "Scotland, UK": "United Kingdom",
    "Singapore": "Singapore",
    "Singapore ": "Singapore",
    "South Africa": "South Africa",
    "South africa": "South Africa",
    "South Africa ": "South Africa",
    "South Korea": "South Korea",
    "Spain": "Spain",
    "spain": "Spain",
    "Spain ": "Spain",
    "Sri Lanka": "Sri Lanka",
    "Sri lanka": "Sri Lanka",
    "Sweden": "Sweden",
    "Sweden ": "Sweden",
    "Switzerland": "Switzerland",
    "SWITZERLAND": "Switzerland",
    "Switzerland ": "Switzerland",
    "Taiwan": "Taiwan",
    "Thailand": "Thailand",
    "The United States": "United States",
    "The US": "United States",
    "Turkey": "Turkey",
    "U.K.": "United Kingdom",
    "U.K. ": "United Kingdom",
    "UK": "United Kingdom",
    "UK ": "United Kingdom",
    "UK (England)": "United Kingdom",
    "United Kingdom": "United Kingdom",
    "United Kingdom ": "United Kingdom",
    "United Kingdom (England)": "United Kingdom",
    "Unites states": "United States",
    "Unites States": "United States",
    "United State": "United States",
    "United State ": "United States",
    "United Stares": "United States",
    "United Stares ": "United States",
    "United Stated": "United States",
    "United States": "United States",
    "United States ": "United States",
    "United States of America": "United States",
    "United States of America ": "United States",
    "United Sates": "United States",
    "United Sates of America": "United States",
    "United Status": "United States",
    "UnitedStates": "United States",
    "United Status": "United States",
    "United Stares ": "United States",
    "Unite States": "United States",
    "USA": "United States",
    "USA ": "United States",
    "Usa": "United States",
    "US": "United States",
    "US ": "United States",
    "U.S.": "United States",
    "U.S. ": "United States",
    "U.S": "United States",
    "U. S.": "United States",
    "U. S. ": "United States",
    "U.S.A": "United States",
    "U.S.A.": "United States",
    "U.S.A. ": "United States",
    "United Statea": "United States",
    "U.S.A ": "United States",
    "ISA": "United States",
    "America": "United States",
    "🇺🇸": "United States",
    "Zimbabwe": "Zimbabwe",
}
data = data[data["country"].isin(country_dict.keys())]
data["country"] = data["country"].replace(country_dict)

# Concatenate city, state (if available), and country
data["full_location"] = data.apply(lambda row: 
                                   f"{row['city']}, {row['us_state']}, {row['country']}" if row["us_state"] 
                                   else f"{row['city']}, {row['country']}", axis=1)


# Print a few rows to verify
print(data[["city", "us_state", "country", "full_location"]].head())
city_counts = data.value_counts('full_location')
# print(len(city_counts))
# print(city_counts.head(1000))

avg_salary_by_location = data.groupby("full_location")["salary"].mean()
avg_salary_by_location = avg_salary_by_location.sort_values()
avg_salary_by_location

In [ ]:
# Gather data from top 150 locations
top_150_locations = data['full_location'].value_counts().head(150).index

# Filter data to only include locations in the top 150
new_data = data.drop(columns=['country', 'city', 'us_state'])
massive_data = new_data[new_data['full_location'].isin(top_150_locations)]

massive_data = pd.get_dummies(massive_data)
massive_data



In [ ]:
# Regression!

# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Goal: Create model to predict salary (USD)
#
# TODO: Define the features (independent variables) X, and target (dependent variable) y
#
# Hint: Drop 2 columns from X

X = massive_data.drop(columns=[#... ]) 
y = massive_data[#... ] 


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Initialize the regression model (Linear Regression in this case)
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
r2 = r2_score(y_test, y_pred)  # R-squared score

# Print the results
print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Optionally, you can check the model's coefficients
print("Model Coefficients:", model.coef_)
print("Model Intercept:", model.intercept_)


In [58]:
massive_data['predicted_salary'] = model.predict(X)


In [ ]:
realtopredicted = massive_data[['salary', 'predicted_salary']]
realtopredicted

In [ ]:
# Group by 'full_location' and calculate the average for the specified columns
grouped_data = new_data.groupby('full_location')[['age', 'years_experience', 'years_in_field', 'salary_usd']].mean().reset_index()
grouped_data_dict = grouped_data.to_dict(orient='records')
# Now, 'grouped_data' contains the average values for each 'full_location'


In [ ]:
from pymongo import MongoClient

# Replace the below URL with your MongoDB cloud connection string
mongo_uri = "your_mongo_connection_string"

# Connect to the MongoDB database
client = MongoClient(mongo_uri)

# Replace 'your_database_name' and 'your_collection_name' with your actual database and collection names
db = client['your_database_name']
collection = db['your_collection_name']

# Insert the data into MongoDB (this will insert all the records in one operation)
collection.insert_many(grouped_data_dict)

print("Data uploaded to MongoDB successfully!")
